In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Función para cargar imágenes desde una carpeta
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

# Inicialización de MediaPipe para detección de manos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Función para extraer puntos clave de una imagen
def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        keypoints = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints.append([lm.x, lm.y, lm.z])
        return np.array(keypoints).flatten()
    return None

# Función para procesar el dataset
def process_dataset(dataset_path):
    data = []
    labels = []
    for label in os.listdir(dataset_path):
        folder = os.path.join(dataset_path, label)
        if os.path.isdir(folder):
            for img in load_images_from_folder(folder):
                keypoints = extract_keypoints(img)
                if keypoints is not None:
                    data.append(keypoints)
                    labels.append(label)
    return np.array(data), np.array(labels)

# Función para aumentar datos directamente en los puntos clave
def augment_keypoints(keypoints):
    augmented = []

    # Rotación
    angle = np.random.uniform(-20, 20)  # Rotar entre -20 y 20 grados
    rotation_matrix = np.array([
        [np.cos(np.radians(angle)), -np.sin(np.radians(angle))],
        [np.sin(np.radians(angle)), np.cos(np.radians(angle))]
    ])
    rotated = np.dot(keypoints.reshape(-1, 3)[:, :2], rotation_matrix.T)

    # Escalado
    scale = np.random.uniform(0.8, 1.2)  # Escalar entre 80% y 120%
    scaled = rotated * scale

    # Traslación
    translation = np.random.uniform(-0.1, 0.1, size=(scaled.shape[0], 2))  # Mover hasta un 10%
    translated = scaled + translation

    # Mantener las coordenadas Z sin cambios
    augmented_keypoints = np.hstack((translated, keypoints.reshape(-1, 3)[:, 2:]))
    augmented.append(augmented_keypoints.flatten())

    return augmented

# Ruta del dataset
dataset_path = "..\\ASL\\asl_dataset\\"

# Procesar el dataset
print("Procesando el dataset...")
X, y = process_dataset(dataset_path)
print(f"Dataset procesado: {X.shape[0]} muestras.")

# Convertir etiquetas a números
unique_labels = sorted(set(y))
label_to_num = {label: idx for idx, label in enumerate(unique_labels)}
y_numeric = np.array([label_to_num[label] for label in y])

# Aumentar datos manualmente
print("Aumentando el dataset...")
augmented_X = []
augmented_y = []

for i in range(len(X)):
    augmented_X.append(X[i])
    augmented_y.append(y_numeric[i])

    # Generar 5 aumentos por muestra
    for _ in range(5):
        augmented_X.append(augment_keypoints(X[i])[0])
        augmented_y.append(y_numeric[i])

X_augmented = np.array(augmented_X)
y_augmented = np.array(augmented_y)
print(f"Dataset aumentado: {X_augmented.shape[0]} muestras.")

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.2, random_state=42)

# Crear el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(unique_labels), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
print("Entrenando el modelo...")
model.fit(X_train, y_train, epochs=40, batch_size=64, validation_data=(X_test, y_test))
print("Entrenamiento completo.")

# Guardar el modelo en formato nativo Keras
model.save("asl_model.keras")
print("Modelo guardado como 'asl_model.keras'.")


In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Función para cargar imágenes desde una carpeta
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

# Inicialización de MediaPipe para detección de manos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Función para extraer puntos clave de una imagen
def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        keypoints = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints.append([lm.x, lm.y, lm.z])
        return np.array(keypoints).flatten()
    return None

# Función para procesar el dataset
def process_dataset(dataset_path):
    data = []
    labels = []
    for label in os.listdir(dataset_path):
        folder = os.path.join(dataset_path, label)
        if os.path.isdir(folder):
            for img in load_images_from_folder(folder):
                keypoints = extract_keypoints(img)
                if keypoints is not None:
                    data.append(keypoints)
                    labels.append(label)
    return np.array(data), np.array(labels)

# Aumento de datos: Rotación, ruido gaussiano y escalado
def augment_keypoints(keypoints):
    augmented = []

    # Rotación
    angle = np.random.uniform(-20, 20)
    rotation_matrix = np.array([
        [np.cos(np.radians(angle)), -np.sin(np.radians(angle))],
        [np.sin(np.radians(angle)), np.cos(np.radians(angle))]
    ])
    rotated = np.dot(keypoints.reshape(-1, 3)[:, :2], rotation_matrix.T)

    # Escalado
    scale = np.random.uniform(0.8, 1.2)
    scaled = rotated * scale

    # Traslación
    translation = np.random.uniform(-0.1, 0.1, size=(scaled.shape[0], 2))
    translated = scaled + translation

    # Añadir ruido gaussiano
    noise = np.random.normal(0, 0.01, translated.shape)
    augmented_keypoints = translated + noise

    # Mantener coordenadas Z sin cambios
    augmented_keypoints = np.hstack((augmented_keypoints, keypoints.reshape(-1, 3)[:, 2:]))
    augmented.append(augmented_keypoints.flatten())

    return augmented

# Ruta del dataset
dataset_path = r"C:\\Users\\danif\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Escritorio\\Universidad\\CURSO 2024-25\\VC\ASL\\asl_dataset\\"

# Procesar el dataset
print("Procesando el dataset...")
X, y = process_dataset(dataset_path)
print(f"Dataset procesado: {X.shape[0]} muestras.")

# Convertir etiquetas a números
unique_labels = sorted(set(y))
label_to_num = {label: idx for idx, label in enumerate(unique_labels)}
num_to_label = {v: k for k, v in label_to_num.items()}
y_numeric = np.array([label_to_num[label] for label in y])

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_numeric, test_size=0.2, random_state=42)

# Crear el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(len(unique_labels), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

# Visualización de curvas de entrenamiento
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión durante el entrenamiento')
plt.legend()

plt.show()

# Guardar el modelo
model.save("asl_model_optimized.keras")
print("Modelo guardado como 'asl_model_optimized.keras'.")


In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Función para cargar imágenes desde una carpeta
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

# Inicialización de MediaPipe para detección de manos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Función para extraer puntos clave de una imagen
def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        keypoints = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints.append([lm.x, lm.y, lm.z])
        return np.array(keypoints)
    return None

# Función para procesar el dataset
def process_dataset(dataset_path):
    data = []
    labels = []
    for label in os.listdir(dataset_path):
        folder = os.path.join(dataset_path, label)
        if os.path.isdir(folder):
            for img in load_images_from_folder(folder):
                keypoints = extract_keypoints(img)
                if keypoints is not None:
                    data.append(keypoints)
                    labels.append(label)
    return np.array(data), np.array(labels)

# Ruta del dataset
dataset_path = r"C:\\Users\\danif\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Escritorio\\Universidad\\CURSO 2024-25\\VC\ASL\\asl_dataset\\"

# Procesar el dataset
print("Procesando el dataset...")
X, y = process_dataset(dataset_path)
print(f"Dataset procesado: {X.shape[0]} muestras.")

# Convertir etiquetas a números
unique_labels = sorted(set(y))
label_to_num = {label: idx for idx, label in enumerate(unique_labels)}
num_to_label = {v: k for k, v in label_to_num.items()}
y_numeric = np.array([label_to_num[label] for label in y])

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_numeric, test_size=0.2, random_state=42)

# Ajustar las dimensiones para LSTM: (n_samples, n_timesteps, n_features)
# X ya tiene la forma (n_samples, n_timesteps=21, n_features=3)
print(f"Forma de entrada para LSTM: {X_train.shape}")

# Crear el modelo LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(len(unique_labels), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

# Visualización de curvas de entrenamiento
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión durante el entrenamiento')
plt.legend()

plt.show()

# Guardar el modelo
model.save("asl_model_lstm.keras")
print("Modelo guardado como 'asl_model_lstm.keras'.")
